<a href="https://colab.research.google.com/github/JohnKeigo/Google-Colaboratory/blob/master/finetune_stable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune Stable Diffusion
Stable Diffusionをファインチューニングするためのnotebookです。<br>
GPU・ハイメモリ設定を前提としています

In [1]:
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e .
!pip install pytorch_lightning tensorboard==2.8 omegaconf einops taming-transformers==0.0.1 clip transformers kornia pip install test-tube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to ./src/taming-transformers
  Running command git clone -q https://github.com/CompVis/taming-transformers.git /content/src/taming-transformers
  Running setup.py develop for taming-transformers
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 706 kB 25.2 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 4.7 MB 23.9 MB/s 
     |████████████████████████████████| 565 kB 61.4 MB/s 
     |████████████████████████████████| 705 kB 50.2 MB/s 
     |████

In [2]:
!git clone https://github.com/rinongal/textual_inversion.git
%cd textual_inversion
%mkdir checkpoint inputs outputs

Cloning into 'textual_inversion'...
remote: Enumerating objects: 233, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 233 (delta 9), reused 8 (delta 8), pack-reused 220
Receiving objects: 100% (233/233), 4.23 MiB | 18.84 MiB/s, done.
Resolving deltas: 100% (81/81), done.
/content/textual_inversion


In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/MyDrive/
!mkdir /content/drive/MyDrive/sd_text_inversion
!mkdir -p /content/drive/MyDrive/sd_text_inversion/Imagesfortraining

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/MyDrive/sd_text_inversion’: File exists


# FineTuning
次に進む前に、以下の手順を行ってください。<br>

- 以下のページからcheckpointを入手し、textual_inversion/checkpoint内に配置してください。<br>
[https://huggingface.co/CompVis/stable-diffusion-v-1-4-original](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original)<br>
ヒント：分からなかったら自前のgoogledriveに配置して、gdownとかでもいいと思います。

- /content/textual_inversion/configs/stable-diffusion/v1-finetune.yamlを開き、
batch_frequencyのサイズを大きく設定し直してください。（example: 9999999）
- 学習させたい画像を/content/textual_inversion/inputsにアップロードしてください。


学習が始まると、毎エポック/content/textual_inversion/logsにcheckpointが保存されます。<br>
学習は適当な所で切り上げてください。

In [3]:
!python3 main.py \
    --base ./configs/stable-diffusion/v1-finetune.yaml \
    -t \
    --actual_resume ./checkpoint/sd-v1-4-full-ema.ckpt \
    -n FINETUNE --gpus 0, --data_root ./inputs

Global seed set to 23
Running on GPUs 0,
Loading model from ./checkpoint/sd-v1-4-full-ema.ckpt
Traceback (most recent call last):
  File "main.py", line 614, in <module>
    model = load_model_from_config(config, opt.actual_resume)
  File "main.py", line 26, in load_model_from_config
    pl_sd = torch.load(ckpt, map_location="cpu")
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 699, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: './checkpoint/sd-v1-4-full-ema.ckpt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "main.py", line 826, in <mo

# Inference
`--embedding_path`と`--config`は適当なパスを割り当ててください<br>
プロンプトを考える際は、「*」という文字が学習させた概念を持っていることに留意して適当に考えてください



In [ ]:
!python3 scripts/stable_txt2img.py \
    --ddim_eta 0.0 --n_samples 1 --n_iter 50 --scale 4.0 --ddim_steps 50 \
    --embedding_path ./logs/HOGEHOGE/checkpoints/HOGEHOGE.pt \
    --ckpt ./checkpoint/sd-v1-4-full-ema.ckpt \
    --config ./logs/HOGEHOGE/configs/*project.yaml \
    --prompt "an image of shinkaimakoto"